In [14]:
# try to add station feature (negative results)

In [1]:
import os
import warnings
warnings. filterwarnings("ignore")

import pandas as pd
import numpy as np
from os import path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import data_process as dp
import draw_map as dm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, plot_confusion_matrix,accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_dir = "./data/indoor_outdoor"
graph_dir = "./graphs"
data_path = os.path.join(data_dir, "new_data_set_with_station_roads_distance_land.csv")
data = pd.read_csv(data_path)
# data.rename(columns={'Unnamed: 0':'timestamp'}, inplace=True)

In [3]:
data['close_road_idx'].fillna(0, inplace=True)
data['close_land_idx'].fillna(0, inplace=True)
data['close_station_idx'].fillna(0, inplace=True)

In [4]:
def add_time_feature_train(data, t=10):
    try:
        data.sort_values(by=['phoneTimestamp'], inplace=True)  
    except: 
        data.sort_values(by=['timestamp'], inplace=True) 
    time_data = data.iloc[0:2]
    time_data['avg_dist'] = 0
    time_data['same_land'] = 0
    time_data['same_road'] = 0
    for i in range(len(data) - t):
        if (len(data[i:i+t+1]['file label'].unique()) == 1):
            temp = data.iloc[i]
            if ((len(data[i:i+t+1]['close_road_idx'].unique()) == 1) and data.iloc[i]['close_road_idx'] != 0):
                temp['same_road'] = 1
            else:
                temp['same_road'] = 0
            if ((len(data[i:i+t+1]['close_land_idx'].unique()) == 1) and data.iloc[i]['close_land_idx'] != 0):
                temp['same_land'] = 1
            else:
                temp['same_land'] = 0  
            temp['avg_dist'] = data[i:i+t+1]['gps_dist'].mean()
            time_data = time_data.append(temp)
        
    time_data = time_data.iloc[2:]  
    time_data['highway'] = time_data['highway'].astype('category')
    time_data['highway_encode'] = time_data['highway'].cat.codes
    time_data['landuse'] = time_data['landuse'].astype('category')
    time_data['landuse_encode'] = time_data['landuse'].cat.codes
    time_data['station'] = time_data['station'].astype('category')
    time_data['station_encode'] = time_data['station'].cat.codes
    
    return time_data

In [5]:
def add_time_feature(data, t=10):
    try:
        data.sort_values(by=['phoneTimestamp'], inplace=True)  
    except: 
        data.sort_values(by=['timestamp'], inplace=True) 
    time_data = data.iloc[0:2]
    time_data['avg_dist'] = 0
    time_data['same_land'] = 0
    time_data['same_road'] = 0
    for i in range(len(data) - t):
        temp = data.iloc[i]
        if ((len(data[i:i+t+1]['close_road_idx'].unique()) == 1) and data.iloc[i]['close_road_idx'] != 0):
            temp['same_road'] = 1
        else:
            temp['same_road'] = 0
        if ((len(data[i:i+t+1]['close_land_idx'].unique()) == 1) and data.iloc[i]['close_land_idx'] != 0):
            temp['same_land'] = 1
        else:
            temp['same_land'] = 0  
        temp['avg_dist'] = data[i:i+t+1]['gps_dist'].mean()
        time_data = time_data.append(temp)
        
    time_data = time_data.iloc[2:]  
    time_data['highway'] = time_data['highway'].astype('category')
    time_data['highway_encode'] = time_data['highway'].cat.codes
    time_data['landuse'] = time_data['landuse'].astype('category')
    time_data['landuse_encode'] = time_data['landuse'].cat.codes
    time_data['station'] = time_data['station'].astype('category')
    time_data['station_encode'] = time_data['station'].cat.codes
    
    return time_data

In [6]:
time_data = add_time_feature_train(data, t=37)

In [7]:
np.unique(time_data['i/o'], return_counts=True) # indoor vs. commuting

(array([0, 1]), array([2067, 2080]))

In [8]:
test = pd.read_csv(path.join(data_dir, "london_unseen_data_with_station_roads_distance_land.csv"))
for i in range(800,1000):
    test.loc[i, 'label'] = 1  # manually re-label data
test.loc[test['label'] == 2 , 'label'] = 1 # commuting 
test['phoneTimestamp'] = test['phoneTimestamp'].astype(str)

test['close_road_idx'].fillna(0, inplace=True)
test['close_land_idx'].fillna(0, inplace=True)
test['close_station_idx'].fillna(0, inplace=True)

In [9]:
test_time_data = add_time_feature(test, t=37)

In [10]:
all_cols = ['bin0', 'bin1', 'bin2', 'bin3', 'bin4', 'bin5', 'bin6', 'bin7', 'bin8',
       'bin9', 'bin10', 'bin11', 'bin12', 'bin13', 'bin14', 'bin15', 'highway_encode', 'distance', 'landuse_encode', 
                  'avg_dist', 'same_road', 'same_land', 'station_encode', 'station_distance']

In [11]:
X = time_data[all_cols] 
y = time_data['i/o']
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)
    
test_x = test_time_data[all_cols]
y = test_time_data['label']

y_test_preds = clf.predict(test_x)
test_time_data['pred_label'] = y_test_preds
    
score = accuracy_score(y, test_time_data['pred_label'])
score

0.7682468021068473

In [12]:
def smooth_label(data, window_size = 10, percentage = 1):
    i = 0    
    while i < (len(data)-window_size-1) :
        if sum(data.iloc[i:i+window_size+1]['pred_label'] == 0) > percentage: 
            data.iloc[i:i+window_size+1]['pred_label'] = 0
        i += window_size
    return data

In [13]:
X = time_data[all_cols] 
y = time_data['i/o']
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)

test_x = test_time_data[all_cols]
y = test_time_data['label']
y_test_preds = clf.predict(test_x)
test_time_data['pred_label'] = y_test_preds

test_time_data = smooth_label(test_time_data, window_size=10, percentage=3)
test_time_data.loc[test_time_data['landuse'] == 'outland', 'pred_label'] = 1
test_time_data.loc[test_time_data['highway'].isin(['tertiary', 'unclassified', 'primary_secondary']), 'pred_label'] = 1


print("Score: {}".format(accuracy_score(y, test_time_data['pred_label'])))
print("F1 Score: {}".format(f1_score(y, test_time_data['pred_label'], average='weighted')))

Score: 0.8081264108352144
F1 Score: 0.807486717857095
